In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [4]:
def process_for_url(text):  # still need to fix hyphen problem
    ignore_capitalize = ["of","the","in"]
    puctuation_without_hyphen = '!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~'
    
    text = text.casefold()
    words =  text.split()
    words = [word.lower() if word in ignore_capitalize or word.startswith('il-') else word.capitalize() for word in words]
    join = str(words).translate(str.maketrans('', '', puctuation_without_hyphen))
    join = hyphen_capitalize(join)

    return join.replace(' ', '+')

def hyphen_capitalize(text):

    l = list(text)
    
    for idx, char in enumerate(l):
        if char == '-':
            l[idx + 1] = l[idx + 1].upper()
            #l.remove(l[idx])

    to_str = ''.join(l)

    return to_str
    

def get_card_price(name,card_set):

    url = f'https://www.mtggoldfish.com/price/{process_for_url(card_set)}/{process_for_url(name)}#online'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    try:
        price = soup.find_all('div')[11].find_all('div')[14]
    except IndexError:
        print(name,card_set,'something went wrong, next cell')
        price = ['0']
    price_num = [da_price.replace('\xa0',' ').replace('$','') for da_price in price][0]
  
    return price_num


    

In [5]:
card_df = pd.read_csv('Cards_Sheet2.csv')

#card_df.applymap()
card_df['Price'] = card_df.apply(lambda row: get_card_price(row.Title,row.Set),axis=1)
card_df.Price = card_df.Price.astype(dtype=float)

card_df['my_value'] = card_df.Quantity * card_df.Price

card_df[['Quantity','Price','my_value']].sum()

Quantity    245.00
Price       575.22
my_value    644.41
dtype: float64

In [6]:
card_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Quantity,170.0,1.441176,0.768891,1.00,1.0000,1.00,2.000,4.00
Price,170.0,3.383647,12.683159,0.22,0.2500,0.37,1.740,150.47
my_value,170.0,3.790647,12.778412,0.22,0.3025,0.60,2.385,150.47


In [7]:
card_df.sort_values(by=['Price'],ascending=False)


,Title,Set,Quantity,Price,my_value
43,metalworker,Urzas Destiny,1,150.47,150.47
112,argothian enchantress,Urzas Saga,1,39.90,39.90
130,lifeline,Urzas Saga,1,33.98,33.98
152,dream halls,Stronghold,1,28.86,28.86
54,repercussion,Urzas Destiny,1,20.67,20.67
...,...,...,...,...,...
159,enfeeblement,Tempest,3,0.24,0.72
156,spike colony,Stronghold,1,0.24,0.24
28,fledgling osprey,Urzas Destiny,1,0.24,0.24
105,divine offering,Mirage,1,0.23,0.23


In [8]:
card_df.sort_values(by=['Price'],ascending=False)[100:].sum()


Title       veiled sentryQuash Zephid's Embrace confiscate...
Set         Urzas SagaUrzas DestinyUrzas SagaUrzas SagaUrz...
Quantity                                                  111
Price                                                   18.27
my_value                                                29.16
dtype: object

In [9]:
card_df.sort_values(by=['Price'],ascending=False)[50:]

,Title,Set,Quantity,Price,my_value
26,false prophet,Urzas Destiny,2,1.17,2.34
140,soul sculptor,Urzas Saga,4,1.13,4.52
33,goblin festival,Urzas Destiny,1,0.92,0.92
157,spike feeder,Stronghold,1,0.84,0.84
8,waiting in the weeds,Classic Sixth Edition,1,0.80,0.80
...,...,...,...,...,...
159,enfeeblement,Tempest,3,0.24,0.72
156,spike colony,Stronghold,1,0.24,0.24
28,fledgling osprey,Urzas Destiny,1,0.24,0.24
105,divine offering,Mirage,1,0.23,0.23


In [10]:
card_df.query('Price < 2.0').sum()

Title       castledivine offeringfeldons caneorder of the ...
Set         Fifth EditionFifth EditionFifth EditionFifth E...
Quantity                                                  192
Price                                                   54.74
my_value                                                79.77
dtype: object

In [11]:
card_df.query('10 <= Price <= 20')

,Title,Set,Quantity,Price,my_value
5,final fortune,Classic Sixth Edition,1,15.63,15.63
18,carnival of souls,Urzas Destiny,2,10.99,21.98
47,opalescence,Urzas Destiny,1,17.31,17.31
99,charisma,Mercadian Masques,1,14.91,14.91
113,barrin master wizard,Urzas Saga,1,17.99,17.99
138,Smokestack,Urzas Saga,1,13.39,13.39
166,time warp,Tempest,1,17.07,17.07


In [12]:
from matplotlib import pyplot as plt

In [1]:
plt.boxplot(card_df.Price.describe(), vert=False)
plt.show()

NameError: name 'plt' is not defined

In [19]:
cheaps = card_df.Price <= 2
mids = (card_df.Price >= 2) & (card_df.Price<150)
spendy = card_df.Price >= 150

In [15]:
card_df[cheaps].sum()

Title       castledivine offeringfeldons caneorder of the ...
Set         Fifth EditionFifth EditionFifth EditionFifth E...
Quantity                                                  192
Price                                                   54.74
my_value                                                79.77
dtype: object

In [18]:
card_df[mids].sum()

Title       Goblin Kingfinal fortuneapprentice necromancer...
Set         Fifth EditionClassic Sixth EditionUrzas Destin...
Quantity                                                   52
Price                                                  370.01
my_value                                               414.17
dtype: object

In [20]:
card_df[spendy]

,Title,Set,Quantity,Price,my_value
43,metalworker,Urzas Destiny,1,150.47,150.47
